In [2]:
import os
import pandas as pd
import hypernetx as hnx
import matplotlib.pyplot as plt

# Define the input directory for processed files and output directory for plots
processed_dir = r'/home/prio/loc_project/Mini_2_ERC_721/labelling_out'
output_dir = r'/home/prio/loc_project/Mini_2_ERC_721/Plots'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Initialize a dictionary to store node degrees over time
node_degrees_over_time = {}

# Process each CSV file in the processed directory
for filename in sorted(os.listdir(processed_dir)):
    if filename.endswith('.csv'):
        file_path = os.path.join(processed_dir, filename)
        
        df = pd.read_csv(file_path)
        
        # Initialize a dictionary to store the hyperedges for the current day
        hyperedges = {}
        
        # Group by 'tokenAddressLabel' to create hyperedges
        for token, group in df.groupby('tokenAddressLabel'):
            hyperedges[token] = set(group['fromLabel']).union(set(group['toLabel']))
        
        # Create the hypergraph for the current day
        H = hnx.Hypergraph(hyperedges)
        
        # Compute degrees of the nodes in the current hypergraph
        node_degrees = {node: H.degree(node) for node in H.nodes}
        
        # Store the degrees in the dictionary
        for node, degree in node_degrees.items():
            if node not in node_degrees_over_time:
                node_degrees_over_time[node] = []
            node_degrees_over_time[node].append(degree)

# Plot the degree changes for each node over time
for node, degrees in node_degrees_over_time.items():
    plt.figure(figsize=(10, 5))
    plt.plot(range(len(degrees)), degrees, marker='o', linestyle='-', color='#1f77b4', label=f'Degree of {node}')
    
    plt.xlabel('Days', fontsize=12)
    plt.ylabel('Degree', fontsize=12)
    plt.title(f'Degree Changes of Node {node} Over Time', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    
    # Save the plot to the output directory
    plt.savefig(os.path.join(output_dir, f'degree_changes_{node}.png'))
    plt.close()

print(f"Plots have been saved in the {output_dir} folder.")


KeyboardInterrupt: 

In [4]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# Define the directory containing the JSON files
directory = '/home/prio/loc_project/Mini_2_ERC_721/degree_hypergraph_everyday'

# Initialize an empty DataFrame to store node degrees over time
degree_df = pd.DataFrame()

# Iterate over each JSON file in the directory
for filename in sorted(os.listdir(directory)):
    if filename.endswith('.json'):
        date = filename.replace('.json', '')  # Extract date from filename
        file_path = os.path.join(directory, filename)
        
        # Read the JSON file
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Convert the JSON data to a DataFrame
        daily_df = pd.DataFrame(list(data.items()), columns=['Node', date])
        daily_df.set_index('Node', inplace=True)
        
        # Add the data to the main DataFrame, aligning by Node
        degree_df = degree_df.join(daily_df, how='outer')

# Replace NaN values with 0 (indicating no degree on that day)
degree_df.fillna(0, inplace=True)

# Convert column names (dates) to datetime objects for better plotting
degree_df.columns = pd.to_datetime(degree_df.columns)

# Plot the degrees of each node over time
plt.figure(figsize=(15, 10))

# Plot each node's degrees over time
for node in degree_df.index:
    plt.plot(degree_df.columns, degree_df.loc[node], marker='o', label=f'Node {node}')

plt.xlabel('Date', fontsize=14)
plt.ylabel('Degree', fontsize=14)
plt.title('Node Degree Changes Daywise', fontsize=16)

# Improve x-axis readability by rotating the labels and formatting dates
plt.xticks(rotation=45, ha='right')

# Add a legend and grid for better readability
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)

# Adjust layout to fit labels and avoid overlap
plt.tight_layout()

# Show the plot
plt.show()
